<a href="https://colab.research.google.com/github/bsse1009/SPL2-nlp/blob/master/full_model/SPL2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import tensorflow as tf
with tf.device('/device:GPU:0'):
  !cp "drive/My Drive/app/models/bidaf.py" .
  !cp "drive/My Drive/app/data/magnitude/wiki-news-300d-1M.magnitude" .
  !cp "drive/My Drive/app/data/magnitude/glove_medium_glove.6B.50d.magnitude.tmp" .
  # !cp "drive/My Drive/app/data/preproccess_squad/dev.answer" .
  # !cp "drive/My Drive/app/data/preproccess_squad/dev.context" .
  # !cp "drive/My Drive/app/data/preproccess_squad/dev.question" .
  # !cp "drive/My Drive/app/data/preproccess_squad/dev.span" .
  # !cp "drive/My Drive/app/data/preproccess_squad/train_3.answer" .
  !cp "drive/My Drive/app/data/preproccess_squad/train_6.context" .
  !cp "drive/My Drive/app/data/preproccess_squad/train_6.question" .
  !cp "drive/My Drive/app/data/preproccess_squad/train_6.span" .
  !cp "drive/My Drive/app/data/preproccess_squad/squad_preproccess.py" .
  !cp "drive/My Drive/app/data/preproccess_squad/download_glove.py" .
  !cp "drive/My Drive/app/layers/BiDAF.py" .
  !cp "drive/My Drive/app/layers/Modelling.py" .
  !cp "drive/My Drive/app/layers/contexual_embedding.py" .
  !cp "drive/My Drive/app/layers/highway_layer.py" .
  !cp "drive/My Drive/app/layers/output.py" .
  !cp "drive/My Drive/app/scripts/magnitude.py" .
  !cp "drive/My Drive/app/data/magnitude/glove-lemmatized.6B.100d.magnitude" .

In [ ]:
!pip install --upgrade pip
!pip install --upgrade wheel
!pip install --upgrade build
!pip install pymagnitude

     |████████████████████████████████| 1.6 MB 4.1 MB/s 
  Attempting uninstall: pip
    Found existing installation: pip 21.1.3
    Uninstalling pip-21.1.3:
      Successfully uninstalled pip-21.1.3
     |████████████████████████████████| 5.4 MB 3.8 MB/s 


In [ ]:
from tensorflow.keras.layers import Input, TimeDistributed, LSTM, Bidirectional, Concatenate, Lambda, Reshape, Add, Activation, Multiply, Dense
from tensorflow.keras.models import Model, load_model
import tensorflow.keras.backend as K
import tensorflow.keras as keras
import numpy as np
from tqdm import tqdm
from highway_layer import Highway
from contexual_embedding import C2VecLayer
import os
from pymagnitude import Magnitude, MagnitudeUtils


In [ ]:
from keras.layers import Layer
import tensorflow as tf
from keras import backend as K
from keras.layers.advanced_activations import Softmax

class SimilarityMatrix(Layer):
  def __init__(self, **kwargs):
    super(SimilarityMatrix, self).__init__(**kwargs)
  
  def build(self, input_shape):
    self.context_shape = input_shape[0]
    self.question_shape = input_shape[1]

    self.kernel = self.add_weight(name="kernel",
                                  shape=(3 * input_shape[0][2], 1),
                                  initializer='uniform',
                                  trainable=True)

    super(SimilarityMatrix, self).build(input_shape)

  def compute_similarity(self, repeated_context_vectors, repeated_query_vectors):

    element_wise_multiply = repeated_context_vectors * repeated_query_vectors
    concatenated_tensor = tf.concat(
    [repeated_context_vectors, repeated_query_vectors, element_wise_multiply], axis=-1)
    dot_product = K.squeeze(K.dot(concatenated_tensor, self.kernel), axis=-1)

    return dot_product

  def build_similarity_matrix(self, context, question):

    num_context_words = K.shape(context)[1]
    num_query_words = K.shape(question)[1]

    context_dim_repeat = K.concatenate([[1, 1], [num_query_words], [1]], 0)
    query_dim_repeat = K.concatenate([[1], [num_context_words], [1, 1]], 0)
    repeated_context_vectors = K.tile(K.expand_dims(context, axis=2), context_dim_repeat)
    repeated_query_vectors = K.tile(K.expand_dims(question, axis=1), query_dim_repeat)
    similarity_matrix = self.compute_similarity(repeated_context_vectors, repeated_query_vectors)
    # similarity_matrix = tf.reshape(similarity_matrix, [self.context_shape[0],self.context_shape[1],self.question_shape[1]])

    return similarity_matrix

  def call(self, x):
    context, question = x
    self.similarity_matrix = self.build_similarity_matrix(context, question)
    return self.similarity_matrix
  
  def compute_output_shape(self, input_shape):
    return (input_shape[0][0],input_shape[0][1],input_shape[1][1])

In [ ]:
from keras.layers import Layer
import tensorflow as tf
from keras import backend as K
from keras.layers.advanced_activations import Softmax

class C2Q_Layer(Layer):
  def __init__(self, **kwargs):
    super(C2Q_Layer, self).__init__(**kwargs)

  def build(self, input_shape):
    super(C2Q_Layer, self).build(input_shape)

  def call(self,x):
    similarity_matrix, question=x
    attention = tf.nn.softmax(similarity_matrix)

    self.U_A=K.sum(K.dot(attention,question),-2)

    return self.U_A

  def compute_output_shape(self, input_shape):
    return self.U_A.shape;

In [ ]:
from keras.layers import Layer
import tensorflow as tf
from keras import backend as K
from keras.layers.advanced_activations import Softmax

class Q2C_Layer(Layer):
  def __init__(self, **kwargs):
    super(Q2C_Layer, self).__init__(**kwargs)

  def build(self, input_shape):
    super(Q2C_Layer, self).build(input_shape)

  def call(self,x):
    similarity_matrix, context=x
    attention = tf.nn.softmax(K.max(similarity_matrix,axis=-1))

    temp=K.expand_dims(K.sum(K.dot(attention,context),-2),1)

    H_A=K.tile(temp,[1,similarity_matrix.shape[1],1])

    return H_A
  
  def compute_output_shape(self, input_shape):
    return self.H_A.shape;

In [ ]:
from keras.layers import Layer
import tensorflow as tf
from keras import backend as K
from keras.layers.advanced_activations import Softmax

class MegaMerge(Layer):
  def __init__(self, **kwargs):
    super(MegaMerge, self).__init__(**kwargs)

  def build(self, input_shape):
    super(MegaMerge, self).build(input_shape)

  def call(self,x):
    context,c2q,q2c=x
    self.G=K.concatenate([context,c2q,context*c2q,context*q2c],axis=-1)

    return self.G;

  def compute_output_shape(self, input_shape):
    return self.G.shape;

In [ ]:
from keras.layers import Layer,LSTM,Bidirectional
import numpy as np
import tensorflow as tf

class ModellingLayer(Layer):
    def __init__(self,**kwargs):
        super(ModellingLayer, self).__init__(**kwargs)

    def build(self, input_shape):
        self.shape=input_shape
  
        self.lstm1 = Bidirectional(LSTM(int(input_shape[2]//8),
                                   activation='tanh',
                                   input_shape=(input_shape[1],input_shape[2]),
                                   return_sequences=True, trainable=True))
        self.lstm2 = Bidirectional(LSTM(int(input_shape[2]//8),
                                   activation='tanh',
                                   input_shape=(input_shape[1], int(input_shape[2]//4)),
                                   return_sequences=True, trainable=True))
        super(ModellingLayer, self).build(input_shape)

    def call(self, x):
        
        self.M1=self.lstm1(x)
        
        self.M2=self.lstm2(self.M1)
        
        self.temp1=tf.concat([x, self.M1], -1)
        self.temp2=tf.concat([x, self.M2], -1)
        
        return self.temp1,self.temp2

    def compute_output_shape(self, input_shape):
        return self.temp1.shape,self.temp2.shape

In [ ]:
from keras.layers import Layer
import tensorflow as tf
from keras import backend as K

class OutputLayer(Layer):
    def __init__(self,**kwargs):
        super(OutputLayer, self).__init__(**kwargs)

    def build(self,input_shape):
  
        self.w1=self.add_weight(name="w1",
                                shape=(input_shape[0][2],),
                                initializer='uniform',
                                trainable=True)
        self.w2=self.add_weight(name="w2",
                                shape=(input_shape[0][2],),
                                initializer='uniform',
                                trainable=True)

        super(OutputLayer, self).build(input_shape)

    def call(self, x):

        answer_span1=tf.tensordot(x[0],tf.transpose(self.w1),1)
        answer_span2=tf.tensordot(x[1], tf.transpose(self.w2), 1)
        
        self.p1=tf.nn.softmax(answer_span1)
        self.p2=tf.nn.softmax(answer_span2)
        
        
        return self.p1,self.p2

    def compute_output_shape(self, input_shape):
        return self.p1.shape,self.p2.shape


In [ ]:
class MagnitudeVectors:

    def __init__(self):
        # print("heloo")
        # base_dir = "F://Pycharm Projects//bidaf-keras-master//data//magnitude"
        glove = Magnitude("glove-lemmatized.6B.100d.magnitude")
        fasttext = Magnitude("wiki-news-300d-1M.magnitude")
        self.vectors = Magnitude(glove,fasttext)

    def load_vectors(self):
        return self.vectors


In [ ]:
from keras.callbacks import *
class BidirectionalAttentionFlow():

    def __init__(self, em_dim, max_passage_length=None, max_query_length=None):
        self.em_dim = em_dim
        self.max_passage_length = max_passage_length
        self.max_query_length = max_query_length
        with tf.device('/device:GPU:0'):
            passage_input = Input(shape=(self.max_passage_length, em_dim), dtype='float32', name="passage_input")
            question_input = Input(shape=(self.max_query_length, em_dim), dtype='float32', name="question_input")
            question_embedding = question_input
            passage_embedding = passage_input

            highway_layers = Highway(name='highway_layer')
            question_embedding = highway_layers(question_embedding)
            passage_embedding = highway_layers(passage_embedding)

            encoder_layer1 = Bidirectional(LSTM(em_dim, activation="tanh", recurrent_activation="sigmoid",
                                                return_sequences=True), name='bidirectional_encoder')
            encoded_question = encoder_layer1(question_embedding)
            # encoder_layer2 = Bidirectional(LSTM(em_dim,
            #                                   return_sequences=True), name='bidirectional_encoder1')
            encoded_passage = encoder_layer1(passage_embedding)

            sim = SimilarityMatrix(name="sm")([encoded_passage, encoded_question])
            c2q = C2Q_Layer(name="c2q")([sim, encoded_question])
            q2c = Q2C_Layer(name="q2c")([sim, encoded_passage])
            megamerge = MegaMerge(name="mega")([encoded_passage, c2q, q2c])
            t1, t2 = ModellingLayer(name="modelling")(megamerge)
            p1, p2 = OutputLayer(name="output")([t1, t2])

            model = Model(inputs=[passage_input, question_input], outputs=[p1, p2])
            model.summary()
            model.compile(loss='categorical_crossentropy', optimizer='sgd', metrics=['accuracy'])
            for i, w in enumerate(model.weights): print(i, w.name)
            self.model = model

    def load_bidaf(self, path):
        custom_objects = {
            "Highway": Highway,
            "SimilarityMatrix": SimilarityMatrix,
            "C2Q_Layer": C2Q_Layer,
            "Q2C_Layer": Q2C_Layer,
            "MegaMerge": MegaMerge,
            "ModellingLayer": ModellingLayer,
            "OutputLayer": OutputLayer
        }

        self.model = load_model(path, custom_objects=custom_objects)

    def predict(self, c, q):
        p1, p2 = self.model.predict(x={"passage_input": c, "question_input": q}, batch_size=1)
        return p1, p2

    def train_model(self, x1, x2, y1, y2, epochs=1):
        with tf.device('/device:GPU:0'):
            filepath="drive/My Drive/app/bidaf_{epoch:03d}.h5"
            checkpoint = ModelCheckpoint(filepath, verbose=1)
            callbacks_list = [checkpoint]
            history = self.model.fit(x={"passage_input": x1, "question_input": x2}, y={"output": y1, "output_1": y2},
                                     batch_size=10,
                                     epochs=epochs, verbose=2,
                                     callbacks=callbacks_list)
            self.model.save('drive/My Drive/app/bidaf250_21.h5')
            return history, self.model

In [ ]:
    with tf.device('/device:GPU:0'):
        context_data = []
        question_data = []
        output_data1 = []
        output_data2 = []
        i = 0
        sample_size = 5000
        vectors = MagnitudeVectors().load_vectors()
        with open("train_6.context", 'r', encoding='utf8') as context_file, \
                open("train_6.question", 'r', encoding='utf8') as question_file, \
                open("train_6.span", 'r', encoding='utf8') as span_file:

            for context, question, span in zip(tqdm(context_file, total=sample_size),
                                               tqdm(question_file, total=sample_size),
                                               tqdm(span_file, total=sample_size)):

                context = [context]
                question = [question]
                passage = [pas.strip() for pas in context]
                cont = []
                for pas in passage:
                    context_tokens = pas.split(" ")
                    cont.append(context_tokens)
                original_passage = [pas.lower() for pas in passage]
                quest = []
                for ques in question:
                    question_tokens = ques.split(" ")
                    quest.append(question_tokens)
                context_batch = vectors.query(cont)
                question_batch = vectors.query(quest)
                pad1 = np.zeros(shape=(1, 250 - len(cont[0]), 400))
                context_batch = np.concatenate((context_batch, pad1), 1)

                pad2 = np.zeros(shape=(1, 20 - len(quest[0]), 400))
                question_batch = np.concatenate((question_batch, pad2), 1)
                answer_span = span.split()
                output1 = np.zeros(shape=(1, 250), dtype=float)
                output2 = np.zeros(shape=(1, 250), dtype=float)
                output1[0][int(answer_span[0])] = 1
                output2[0][int(answer_span[1])] = 1
                # context_batch = tf.convert_to_tensor(context_batch, tf.float32)
                # question_batch = tf.convert_to_tensor(question_batch, tf.float32)
                # output1 = tf.convert_to_tensor(output1, tf.float32)
                # output2 = tf.convert_to_tensor(output2, tf.float32)

                context_data.append(context_batch)
                question_data.append(question_batch)
                output_data1.append(output1)
                output_data2.append(output2)
                # bidef.train_model(context_batch, question_batch, output, 5)
                i = i + 1
                if i == sample_size:
                    break
                print(i)

        context_data = np.array(context_data)
        context_data = np.reshape(context_data, (sample_size, 250, 400))
        question_data = np.array(question_data)
        question_data = np.reshape(question_data, (sample_size, 20, 400))
        output_data1 = np.array(output_data1)
        output_data1 = np.reshape(output_data1, (sample_size, 250))
        output_data2 = np.array(output_data2)
        output_data2 = np.reshape(output_data2, (sample_size, 250))

In [ ]:

bidef = BidirectionalAttentionFlow(400, 250, 20)
bidef.load_bidaf('drive/My Drive/app/bidaf250_20.h5')
print("loaded")
bidef.train_model(context_data, question_data, output_data1, output_data2, 5)

In [ ]:
vectors = MagnitudeVectors().load_vectors()


def processForModel(context, question, span):
        context = [context]
        question = [question]
        passage = [pas.strip() for pas in context]
        cont = []
        for pas in passage:
            context_tokens = pas.split(" ")
            cont.append(context_tokens)
        original_passage = [pas.lower() for pas in passage]
        quest = []
        for ques in question:
            question_tokens = ques.split(" ")
            quest.append(question_tokens)
        context_batch = vectors.query(cont)
        question_batch = vectors.query(quest)
        pad1 = np.zeros(shape=(1, 250 - len(cont[0]), 400))
        context_batch = np.concatenate((context_batch, pad1), 1)

        pad2 = np.zeros(shape=(1, 20 - len(quest[0]), 400))
        question_batch = np.concatenate((question_batch, pad2), 1)
        answer_span = span.split()
        output1 = np.zeros(shape=(1, 250), dtype=float)
        output2 = np.zeros(shape=(1, 250), dtype=float)
        output1[0][int(answer_span[0])] = 1
        output2[0][int(answer_span[1])] = 1
        # context_batch = tf.convert_to_tensor(context_batch, tf.float32)
        # question_batch = tf.convert_to_tensor(question_batch, tf.float32)
        # output1 = tf.convert_to_tensor(output1, tf.float32)
        # output2 = tf.convert_to_tensor(output2, tf.float32)

        return context_batch, question_batch, output1, output2


In [ ]:
bidef = BidirectionalAttentionFlow(400, 250, 20)
bidef.load_bidaf('drive/My Drive/app/bidaf250_15.h5')
context = "in four months , aonuma 's team managed to present realistic horseback riding , [ l ] which nintendo later revealed to the public with a trailer at electronic entertainment expo 2004 . the game was scheduled to be released the next year , and was no longer a follow-up to the wind waker ; a true sequel to it was released for the nintendo ds in 2007 , in the form of phantom hourglass . miyamoto explained in interviews that the graphical style was chosen to satisfy demand , and that it better fit the theme of an older incarnation of link . the game runs on a modified the wind waker engine ."
question = "where did nintendo preview the horseback riding feature ?"
context = 'from the beginning of 2014 , madonna began to make multiple media appearances . she appeared at the 56th annual grammy awards in january 2014 , performing " open your heart " alongside rappers macklemore & ryan lewis and singer mary lambert , who sang their single " same love " , as 33 couples were wed onstage , officiated by queen latifah . days later , she joined singer miley cyrus on her mtv unplugged special , singing a mash-up of " do n\'t tell me " and cyrus \' single " we ca n\'t stop " ( 2013 ) . she also extended her business ventures and in february 2014 the singer premiered mdna skin , a range of skin care products , in tokyo , japan . after visiting her hometown of detroit during may 2014 , madonna decided to contribute funds to three of the city \'s organizations , to help eliminate poverty from there . the singer released a statement saying that she was inspired by their work , adding that " it was obvious to me that i had to get involved and be part of the solution to help detroit recover " .'
question="which singer did she join on mtv unplugged to sing ?"
span = "70 71"
c, q, s1, s2 = processForModel(context, question, span)
p1, p2 = bidef.predict(c, q)

print(np.amax(p1, axis=1))
print(np.amax(p2, axis=1))

print(p1, p2)
print(s1, s2)


Model: "model_7"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
passage_input (InputLayer)      [(None, 250, 400)]   0                                            
__________________________________________________________________________________________________
question_input (InputLayer)     [(None, 20, 400)]    0                                            
__________________________________________________________________________________________________
highway_layer (Highway)         multiple             320800      question_input[0][0]             
                                                                 passage_input[0][0]              
__________________________________________________________________________________________________
bidirectional_encoder (Bidirect multiple             2563200     highway_layer[0][0]        